## Exporting Climate Time-Series as COGs

In [2]:
import ee

In [3]:
cloud_project = 'spatialthoughts'

try:
    ee.Initialize(project=cloud_project)
except:
    ee.Authenticate()
    ee.Initialize(project=cloud_project)

#### Pre-Processing

In [4]:
terraclimate = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE')
# Soil Moisture
soil = terraclimate.select('soil')

def scale(image):
  return image.multiply(0.1) \
    .copyProperties(image,['system:time_start'])

soilScaled = soil.map(scale)

filtered = soilScaled \
  .filter(ee.Filter.date('2023-01-01', '2025-01-01')) \

image_ids = filtered.aggregate_array('system:index').getInfo()
print('Total images: ', len(image_ids))

Total images:  12


#### Export Images

In [6]:
# Change the bucket name you a bucket where you may write access
gcs_bucket_name = 'spatialthoughts-public-data'
gcs_bucker_folder = 'terraclimate'

In [8]:
for i, image_id in enumerate(image_ids):
    exportImage = ee.Image(filtered.filter(ee.Filter.eq('system:index', image_id)).first())
    geometry = ee.Algorithms.GeometryConstructors.BBox(-180, -90, 180, 90)
    task_name = f'soil_moisture_{image_id}'

    task = ee.batch.Export.image.toCloudStorage(**{
            'image': exportImage,
            'description': f'Image_Export_{task_name}',
            'fileNamePrefix': f'{gcs_bucker_folder}/{task_name}',
            'bucket': gcs_bucket_name,
            'region': geometry,
            'maxPixels': 1e10,
            'formatOptions': {'cloudOptimized': True},
        })
    task.start()
    print(f'Started Task: {task_name}')

Started Task: soil_moisture_202301
Started Task: soil_moisture_202302
Started Task: soil_moisture_202303
Started Task: soil_moisture_202304
Started Task: soil_moisture_202305
Started Task: soil_moisture_202306
Started Task: soil_moisture_202307
Started Task: soil_moisture_202308
Started Task: soil_moisture_202309
Started Task: soil_moisture_202310
Started Task: soil_moisture_202311
Started Task: soil_moisture_202312
